<a href="https://colab.research.google.com/github/CarolinaBRangel/ML-Final-Project/blob/main/Merge_dos_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import lightgbm as lgb
import csv
from sklearn import preprocessing, metrics
from sklearn.preprocessing import LabelEncoder
import gc
import os
from tqdm import tqdm
from scipy.sparse import csr_matrix

In [ ]:
files.upload()

! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c m5-forecasting-accuracy
!unzip \*.zip  && rm *.zip

Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
 48% 22.0M/45.8M [00:00<00:00, 124MB/s] 
100% 45.8M/45.8M [00:00<00:00, 164MB/s]
Archive:  m5-forecasting-accuracy.zip
replace calendar.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: none
replace sales_train_evaluation.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace sales_train_validation.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace sell_prices.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no


In [ ]:
sell_prices = pd.read_csv("sell_prices.csv", index_col=False) 
sales_eval = pd.read_csv("sales_train_evaluation.csv", index_col=False)
calendar = pd.read_csv("calendar.csv", index_col=False)

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: 
        col_type = df[col].dtypes
        if col_type in numerics:
          c_min = df[col].min()
          c_max = df[col].max()
          if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
          else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
def read_data():
    print('Reading files...')

    calendar = pd.read_csv('calendar.csv')
    calendar = reduce_mem_usage(calendar)
    print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
    
    sell_prices = pd.read_csv('sell_prices.csv')
    sell_prices = reduce_mem_usage(sell_prices)
    print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))
    
    sales_eval = pd.read_csv('sales_train_evaluation.csv')
    print('Sales train evaluation has {} rows and {} columns'.format(sales_eval.shape[0], sales_eval.shape[1]))
    
    return calendar, sell_prices, sales_eval 

In [ ]:
import IPython

def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

In [ ]:
calendar, sell_prices, sales_eval  = read_data()

Reading files...
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Sales train evaluation has 30490 rows and 1947 columns


In [ ]:
def encode_categorical(df, cols):
    
    for col in cols:
        # Leave NaN as it is.
        le = LabelEncoder()
        not_null = df[col][df[col].notnull()]
        df[col] = df[col].fillna('nan')
        df[col] = pd.Series(le.fit_transform(df[col]), index=df.index)
        return df

calendar = encode_categorical(
    calendar, ["event_name_1", "event_type_1", "event_name_2", "event_type_2"]
).pipe(reduce_mem_usage)

sales_eval = encode_categorical(
    sales_eval, ["item_id", "dept_id", "cat_id", "store_id", "state_id"],
).pipe(reduce_mem_usage)

sell_prices = encode_categorical(sell_prices, ["item_id", "store_id"]).pipe(
    reduce_mem_usage
)

Mem. usage decreased to  0.11 Mb (10.8% reduction)
Mem. usage decreased to 95.96 Mb (78.8% reduction)
Mem. usage decreased to 91.34 Mb (30.0% reduction)


In [ ]:
sales_eval = sales_eval.drop(['id', 'dept_id',], 1) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [ ]:
sales_eval

,item_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,1437,HOBBIES,CA_1,CA,0,0,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,1438,HOBBIES,CA_1,CA,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,1439,HOBBIES,CA_1,CA,0,0,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,1440,HOBBIES,CA_1,CA,0,0,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,1441,HOBBIES,CA_1,CA,0,0,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,1432,FOODS,WI_3,WI,0,0,2,2,0,3,...,1,0,3,0,1,1,0,0,1,1
30486,1433,FOODS,WI_3,WI,0,0,0,0,0,5,...,0,0,0,0,0,0,1,0,1,0
30487,1434,FOODS,WI_3,WI,0,6,0,2,2,4,...,0,0,1,2,0,1,0,1,0,2
30488,1435,FOODS,WI_3,WI,0,0,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [ ]:
calendar = calendar.drop(['weekday','event_name_1', 'event_name_2', 'year', 'month'], 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [ ]:
calendar = calendar.fillna(value = 0)

In [ ]:
#calendar['d'] = calendar['d'].map(lambda x: x.lstrip('d_'))

In [ ]:
calendar

,date,wm_yr_wk,wday,d,event_type_1,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,1,d_1,0,0,0,0,0
1,2011-01-30,11101,2,d_2,0,0,0,0,0
2,2011-01-31,11101,3,d_3,0,0,0,0,0
3,2011-02-01,11101,4,d_4,0,0,1,1,0
4,2011-02-02,11101,5,d_5,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,5,d_1965,0,0,0,1,1
1965,2016-06-16,11620,6,d_1966,0,0,0,0,0
1966,2016-06-17,11620,7,d_1967,0,0,0,0,0
1967,2016-06-18,11621,1,d_1968,0,0,0,0,0


In [ ]:
l=[]
for i in range(1,1914):
  l.append("d_"+str(i))
train = pd.melt(sales_eval,id_vars=['item_id', 'cat_id', 'store_id', 'state_id'],\
                 value_vars=l,var_name="d",value_name="sales")

In [ ]:
#train['d'] = train['d'].map(lambda x: x.lstrip('d_'))

In [ ]:
train = train.merge(calendar,on='d',copy=False)
train = train.merge(sell_prices,on=["store_id", "item_id", "wm_yr_wk"],copy=False) 

In [ ]:
train

,item_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,wday,event_type_1,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,1444,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,1,0,0,0,0,0,0.459961
1,1444,HOBBIES,CA_1,CA,d_2,15,2011-01-30,11101,2,0,0,0,0,0,0.459961
2,1444,HOBBIES,CA_1,CA,d_3,0,2011-01-31,11101,3,0,0,0,0,0,0.459961
3,1444,HOBBIES,CA_1,CA,d_4,0,2011-02-01,11101,4,0,0,1,1,0,0.459961
4,1444,HOBBIES,CA_1,CA,d_5,0,2011-02-02,11101,5,0,0,1,0,1,0.459961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46027952,1434,FOODS,WI_3,WI,d_1913,0,2016-04-24,11613,2,0,0,0,0,0,3.980469
46027953,1435,FOODS,WI_3,WI,d_1912,1,2016-04-23,11613,1,0,0,0,0,0,1.280273
46027954,1435,FOODS,WI_3,WI,d_1913,3,2016-04-24,11613,2,0,0,0,0,0,1.280273
46027955,1436,FOODS,WI_3,WI,d_1912,0,2016-04-23,11613,1,0,0,0,0,0,1.000000


In [ ]:
from google.colab import drive
drive.mount('drive', force_remount=True)

Mounted at drive


In [ ]:
train.to_csv('train.csv')
!cp train.csv "drive/My Drive/"

KeyboardInterrupt: ignored